In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

### Data

In [ ]:
# Load the financial news dataset and do some basic transformations
import pandas as pd

df = pd.read_csv('/kaggle/input/financial-news/articles.csv')
df_links = pd.read_csv('/kaggle/input/financial-news/links.csv')
df = df.merge(df_links, left_on ='source_url', right_on= 'link')
# # Reduce the number of observations
# df.drop_duplicates(subset='text', inplace = True)
# # Draw random sample of 20%
# df = df.sample(frac=0.1, random_state = 42)
df['date'] = pd.to_datetime(df.date).dt.tz_localize("EST")
df['date_utc'] = df.date.dt.tz_convert("UTC")
df['year'] = df['date_utc'].dt.year
df['month'] = df['date_utc'].dt.month
df['release']= df['date'].dt.date
df.dropna(inplace = True)
df.rename(columns={'instrument':'ticker'}, inplace=True)
df = df[df['year'] >= 2016]

#### Data Exploration

In [ ]:
df.columns

In [ ]:
print('Number of unique articles: ' + str(len(df.text.unique())))

In [ ]:
# get random article
import random
random.seed(5)

for i in random.sample(range(0, len(df)-1),1):
    print(df.text[i])
    print('___________________________________________________________________________________')

In [ ]:
# check by ticker
df.ticker.value_counts()

In [ ]:
# check by source
df.source.value_counts()

In [ ]:
import matplotlib.pyplot as plt
df['release'] = pd.to_datetime(df['release'])
headlines_count = df.groupby('date').size()

plt.figure(figsize=(12,6))
headlines_count.plot()
plt.title('Number of News Headlines per Day')
plt.xlabel('Date')
plt.ylabel('Number of Headlines')
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# set ticker of interest
tickerSymbol = "US500"

In [ ]:
# filter df for ticker
df = df[df['ticker']==tickerSymbol]

In [ ]:
df.drop_duplicates(subset='text', keep='first', inplace=True)

In [ ]:
# check by ticker
df.ticker.value_counts()

In [ ]:
# check by year
df.year.value_counts()

In [ ]:
display(min(df.release))
display(max(df.release))

In [ ]:
df = df[['release','headline', 'text']]
# df.rename(columns={'release':'date'}, inplace=True)

In [ ]:
df.to_csv('/kaggle/working/SP500_news.csv')

#### Financial Data

In [ ]:
!pip install yfinance
import yfinance as yf

In [ ]:
data = []
tickerSymbol = '^GSPC'
start = min(df['release'])
end = max(df['release'])
ticker = yf.Ticker(tickerSymbol)
fin_data = ticker.history(start=start, end = end)

In [ ]:
fin_data = fin_data.reset_index()[['Date','Open', 'Close', 'High', 'Low', 'Volume']]

In [ ]:
# Convert 'date' column to datetime type
fin_data['Date'] = pd.to_datetime(fin_data['Date'])

# Format 'date' column to 'YYYY-MM-DD'
fin_data['Date'] = fin_data['Date'].dt.strftime('%Y-%m-%d')
fin_data['Date'] = pd.to_datetime(fin_data['Date'])
fin_data.rename(columns={'Date':'date'}, inplace=True)

# Convert 'date' column to datetime type
df['release'] = pd.to_datetime(df['release'])

# Format 'date' column to 'YYYY-MM-DD'
df['release'] = df['release'].dt.strftime('%Y-%m-%d')
df['release'] = pd.to_datetime(df['release'])

In [ ]:
fin_data.to_csv('/kaggle/working/SP500_stock.csv')

In [ ]:
# Convertadd_suffix the trading days to a set for faster lookup
trading_days_set = set(fin_data['date'].dt.date)
max_trading_date = fin_data['date'].max().date()

def next_trading_day(x):
    # Convert the date to a python date object for compatibility with the set
    x_date = x.date()
    # Keep advancing a day until it's a trading day or exceeds the max trading date
    while x_date not in trading_days_set and x_date <= max_trading_date:
        x += pd.DateOffset(days=1)
        x_date = x.date()
    return pd.Timestamp(x_date) if x_date <= max_trading_date else pd.NaT  # Return NaT if date is out of bounds

df['date'] = df['release'].apply(next_trading_day)


In [ ]:
df.dropna(inplace = True)

In [ ]:
df

In [ ]:
df_input = fin_data.merge(df,on='date', how= 'outer')

In [ ]:
df_input.groupby("date")["Close"].count()

In [ ]:
import matplotlib.pyplot as plt
df_input['date'] = pd.to_datetime(df_input['date'])
headline_counts = df_input.groupby("date")["headline"].count()

plt.figure(figsize=(12,6))
headline_counts.plot()
plt.title('Number of News Headlines per Day')
plt.xlabel('Date')
plt.ylabel('Number of Headlines')
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
df_input.to_csv('/kaggle/working/SP500_all.csv')